### 0. Import

In [3]:
from transformers import AutoTokenizer
from customized import CustomizedGPTJForCausalLM
import torch

### 1. Loading Models

In [4]:
model = CustomizedGPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6b", torch_dtype=torch.bfloat16).cuda()
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6b", padding_side="left")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<|PAD|>'})
    model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at EleutherAI/gpt-j-6b were not used when initializing CustomizedGPTJForCausalLM: ['transformer.h.0.attn.masked_bias', 'transformer.h.12.attn.masked_bias', 'transformer.h.22.attn.bias', 'transformer.h.18.attn.masked_bias', 'transformer.h.24.attn.bias', 'transformer.h.13.attn.bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.21.attn.bias', 'transformer.h.10.attn.bias', 'transformer.h.16.attn.masked_bias', 'transformer.h.26.attn.bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.17.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.20.attn.masked_bias', 'transformer.h.27.attn.bias', 'transformer.h.26.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.24.attn.masked_bias', 'transformer.h.25.attn.bias', 'transformer.h.4.attn.bias', 'tr

### 2. Information Unveilling

In [10]:
# Threshold for confidence score of the generated content
threshold_confidence = 0.5

# The maximum generated token from the hidden state
max_length = 5

# Sub-layer in [0, 27] to extract hidden state from
sublayer = 21

# Specify the prefix that is best at unveiling the information at the subsequent n token
best_prediction_at = 1

# Utterance to extract hidden state from, will always extract the last token hidden states
utterance = "Specifically, we train GPT-3, an"

# Loading the corresponding trained prefix 
model.load_prefix(f"./results/training/layer{sublayer}to27_tk{best_prediction_at}/soft_prefix.pt")
model.eval()

# Tokenize the utterance
input_ids = tokenizer(utterance, return_tensors="pt").input_ids.cuda()

# Get the original hidden state from the sublayer
model.unprefix_model()
original_hidden_state = model(input_ids, output_hidden_states=True).hidden_states[sublayer][:, -1, :]


TypeError: object of type 'NoneType' has no len()

### 3. Information Plot